In [1]:
#Imports
# type: ignore
import os
from dotenv import load_dotenv
import pandas as pd

from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import DataFrameLoader
from langchain.docstore.document import Document

In [2]:
# Import des variables d'environnement (.env)

load_dotenv()

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY_4")
azure_openai_api_endpoint = os.getenv("AZURE_OPENAI_API_ENDPOINT_4")
azure_deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME_4")

In [3]:
# Initialisation du modèle d'Embeddings

embedding_model = AzureOpenAIEmbeddings(
    openai_api_key=azure_openai_api_key,
    deployment="text-embedding-3-large",
    azure_endpoint=azure_openai_api_endpoint,
    openai_api_version="2023-05-15",
    chunk_size=500
)

/tmp/ipykernel_19784/1634952500.py:3: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureOpenAIEmbeddings``.
  embedding_model = AzureOpenAIEmbeddings(


In [4]:
# Import du dataset 

gutenberg_data = pd.read_csv("Data/gutenberg_ebooks.csv")

In [5]:
gutenberg_data.tail()

,Ebook ID,Author,Title,Credits,Summary,Language,LoC Class,Subject,Subject_2,Subject_3,Subject_4,Category,EBook-No.,Release Date,Most Recently Updated,Copyright Status,Downloads
19924,19996,Various,"The Atlantic Monthly, Volume 16, No. 96, Octob...","Produced by Joshua Hutchinson, Josephine Paolu...","""The Atlantic Monthly, Volume 16, No. 96, Octo...",English,AP: General Works: Periodicals,American periodicals,NaN,NaN,NaN,Text,19996,"Dec 2, 2006",NaN,Public domain in the USA.,88 downloads in the last 30 days.
19925,19997,"Ober, Frederick A. (Frederick Albion), 1849-1913",Amerigo Vespucci,"Produced by Suzanne Shell, Richard J. Shiffer ...","""Amerigo Vespucci"" by Frederick A. Ober is a h...",English,E011: History: America: America,"Vespucci, Amerigo, 1451-1512",NaN,NaN,NaN,Text,19997,"Dec 3, 2006",NaN,Public domain in the USA.,145 downloads in the last 30 days.
19926,19998,"Allen, Lewis Falley, 1800-1890",Rural ArchitectureBeing a Complete Description...,"Produced by Louise Hope, Steven Giacomelli and...","""Rural Architecture"" by Lewis Falley Allen is ...",English,NA: Fine Arts: Architecture,Domestic animals,"Architecture, Domestic",NaN,NaN,Text,19998,"Dec 3, 2006",NaN,Public domain in the USA.,232 downloads in the last 30 days.
19927,19999,"Trowbridge, J. T. (John Townsend), 1827-1916",The Drummer Boy,Produced by David Edwards and the Online Distr...,"""The Drummer Boy"" by J. T. Trowbridge is a his...",English,PZ: Language and Literatures: Juvenile belles ...,"United States -- History -- Civil War, 1861-18...",NaN,NaN,NaN,Text,19999,"Dec 3, 2006",NaN,Public domain in the USA.,76 downloads in the last 30 days.
19928,20000,"Verne, Jules, 1828-1905",Twenty Thousand Leagues Under the Sea,NaN,NaN,English,PQ: Language and Literatures: Romance literatu...,Science fiction,Submarines (Ships) -- Fiction,Sea stories,Underwater exploration -- Fiction,Sound,20000,"Sep 1, 2007","Jan 30, 2021",Public domain in the USA.,515 downloads in the last 30 days.


In [6]:
gutenberg_data.shape

(19929, 17)

In [7]:
print(gutenberg_data.isnull().sum())

Ebook ID                     0
Author                     874
Title                        0
Credits                   1563
Summary                    819
Language                     0
LoC Class                    8
Subject                      9
Subject_2                 8253
Subject_3                13595
Subject_4                16257
Category                     0
EBook-No.                    0
Release Date                 0
Most Recently Updated     1821
Copyright Status             0
Downloads                    0
dtype: int64


In [8]:
gutenberg_data = gutenberg_data.dropna(subset=["Summary", "Title"])

In [9]:
gutenberg_data.shape

(19110, 17)

In [10]:
metadata_columns = ["Title", "Author", "Language", "Release Date", "Subject", "Subject_2", "Subject_3", "Subject_4"]

gutenberg_data[metadata_columns] = gutenberg_data[metadata_columns].fillna('')


In [11]:
# type: ignore
from langchain.schema import Document

documents = []

for idx, row in gutenberg_data.iterrows():

    # On va combiner l'ensemble des sujets en un seul bloc
    subjects = ', '.join([row[col] for col in ["Subject", "Subject_2", "Subject_3", "Subject_4"] if row[col]])
    
    # Création du dictionnaire des métadonnées avec rassemblement de toutes les informations clé
    metadata = {
        "Title": row["Title"],
        "Author": row["Author"],
        "Language": row["Language"],
        "Release Date": row["Release Date"],
        "Subjects": subjects
    }
    
    document = Document(page_content=row["Summary"], metadata=metadata)
    documents.append(document)

In [15]:
# type: ignore

# Utilisation avec FAISS (= moteur de stockage vectoriel)
# Objectif : Permettre une recherche de similarité efficace

# Le kernel crash lors de la création de l'index vectoriel :
# TEST 1 : Toujours avec FAISS mais avec un traitement par lots (batch)
# TEST 2 SI PERSISTENCE ERREURS AVEC FAIS : Utiliser Chroma


from langchain.vectorstores import FAISS

# Mise en place du traitement par lot : 
batch_size = 500

vectorstore = None

"""index = VectorstoreIndexCreator(
    embedding=embedding_model,
    vectorstore_cls=FAISS

    # Création de l'index vectoriel
).from_documents(documents)"""

# Traitement des documents par lots
for i in range(0, len(documents), batch_size):
    batch_docs = documents[i:i+batch_size]
    print(f"Traitement du lot {i//batch_size + 1}/{(len(documents) + batch_size - 1)//batch_size}")

    if vectorstore is None:
        vectorstore = FAISS.from_documents(batch_docs, embedding_model)
    else:
        new_vectorstore = FAISS.from_documents(batch_docs, embedding_model)
        vectorstore.merge_from(new_vectorstore)

# Enregistrement de l'index complet
vectorstore.save_local("book_index")

Traitement du lot 1/39
Traitement du lot 2/39
Traitement du lot 3/39
Traitement du lot 4/39
Traitement du lot 5/39
Traitement du lot 6/39
Traitement du lot 7/39
Traitement du lot 8/39
Traitement du lot 9/39
Traitement du lot 10/39
Traitement du lot 11/39
Traitement du lot 12/39
Traitement du lot 13/39
Traitement du lot 14/39
Traitement du lot 15/39
Traitement du lot 16/39
Traitement du lot 17/39
Traitement du lot 18/39
Traitement du lot 19/39
Traitement du lot 20/39
Traitement du lot 21/39
Traitement du lot 22/39
Traitement du lot 23/39
Traitement du lot 24/39
Traitement du lot 25/39
Traitement du lot 26/39
Traitement du lot 27/39
Traitement du lot 28/39
Traitement du lot 29/39
Traitement du lot 30/39
Traitement du lot 31/39
Traitement du lot 32/39
Traitement du lot 33/39
Traitement du lot 34/39
Traitement du lot 35/39
Traitement du lot 36/39
Traitement du lot 37/39
Traitement du lot 38/39
Traitement du lot 39/39


In [12]:
print(f"Nombre de docs créés : {len(documents)}")

Nombre de docs créés : 19110


In [23]:
# type: ignore
# Import du fichier enregistré précédemment

from langchain.vectorstores import FAISS

vectorstore = FAISS.load_local(
    "book_index",
    embeddings=embedding_model,
    # Ligne ci dessous à ajouter sinon erreur lors du lancement de la cellule
    allow_dangerous_deserialization=True
)

In [24]:
# Création du retriever
retriever = vectorstore.as_retriever()

In [25]:
# type: ignore
# Initialisation du modèle de langage conversationnel Azure OpenAI

from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
                        api_key=azure_openai_api_key,
                        api_version="2023-12-01-preview",
                        azure_endpoint=azure_openai_api_endpoint,
                        model=azure_deployment_name,
                        temperature=0
                        )

In [26]:
# type: ignore
# Utilisation d'un prompt personnalisé

from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """
Vous êtes un assistant expert en littérature.

Utilisez les informations suivantes sur les livres pour répondre à la question de manière précise et détaillée.

Informations :
{context}

Question :
{question}

Réponse :
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [28]:
# Création de la chaine de Questions / Réponses

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    # importer le prompt personnalisé
    chain_type_kwargs={"prompt": PROMPT}
)

In [29]:
#Exemple d'utilisation 1
question = "Quels sont les thèmes principaux du livre 'Les Misérables' ?"
answer = qa_chain.run(question)
print(answer)

Les thèmes principaux de "Les Misérables" de Victor Hugo incluent :

1. **Justice et Morale** : Le roman explore la complexité de la justice et de la morale à travers les expériences de ses personnages, notamment Jean Valjean, qui cherche à échapper à son passé criminel et à se racheter.

2. **Rédemption** : Le parcours de Jean Valjean est central, illustrant la possibilité de rédemption et de transformation personnelle malgré un passé difficile.

3. **Lutte entre le Bien et le Mal** : Les personnages sont souvent confrontés à des choix moraux, symbolisant la lutte constante entre le bien et le mal.

4. **Injustice Sociale et Pauvreté** : Hugo met en lumière la misère des classes défavorisées et critique les inégalités sociales de la France post-révolutionnaire.

5. **Compassion et Charité** : Incarnées par des personnages comme l'évêque Myriel, ces valeurs sont présentées comme des forces positives capables de changer des vies.

6. **Révolution et Insurrection** : Le roman se déroule 

In [30]:
# Exemple d'utilisation 2
question = "Quels sont les principaux livres de Shakespeare ?"
answer = qa_chain.run(question)
print(answer)

Les principaux livres de Shakespeare incluent une variété de pièces de théâtre et de poèmes qui ont marqué la littérature. Voici quelques-unes de ses œuvres les plus notables :

1. **Tragédies :**
   - "Roméo et Juliette" : Une tragédie sur l'amour interdit entre deux jeunes amants issus de familles rivales à Vérone.
   - "Macbeth" : L'histoire d'un noble écossais dont l'ambition démesurée le conduit à la ruine.
   - "Hamlet" : Un drame complexe sur la vengeance et la folie, centré sur le prince du Danemark.
   - "Othello" : Une exploration des thèmes de la jalousie et de la trahison.

2. **Comédies :**
   - "Peines d'amour perdues" : Une comédie sur les vœux de célibat de jeunes nobles mis à l'épreuve par l'arrivée de la princesse de France.
   - "Le Songe d'une nuit d'été" : Une pièce mêlant amour et magie dans une forêt enchantée.
   - "La Mégère apprivoisée" : Une comédie sur le mariage et la transformation des caractères.

3. **Histoires :**
   - "Henri V" : Une pièce historique s

In [31]:
# Exemple d'utilisation 3
question = "Quels sont les personnages principaux de 'The Drumer Boy' et de quel sujet traite ce livre ?"
answer = qa_chain.run(question)
print(answer)

Les personnages principaux de "The Drummer Boy" de J. T. Trowbridge sont Frank Manly et sa famille, notamment sa mère, sa sœur Helen, et sa plus jeune sœur Hattie. Le livre traite du désir de Frank de s'engager comme tambour lors de la guerre civile américaine, malgré les objections émotionnelles de sa famille. Le récit explore des thèmes tels que le patriotisme, le courage et le sacrifice personnel dans le contexte d'une nation en guerre. L'histoire commence par une conversation émotive entre Frank et sa famille un soir d'automne en 1861, où il exprime sa volonté de se battre pour l'Union, malgré les inquiétudes de sa mère et les supplications de sa sœur Hattie.


In [32]:
# Exemple d'utilisation autre que sur les livres 
question = "Quel temps fera-t-il demain ?"
answer = qa_chain.run(question)
print(answer)

Je suis désolé, mais je ne peux pas prédire la météo. Cependant, je peux vous aider avec des informations sur les livres mentionnés ou d'autres questions littéraires.


#### NEXT STEPS : 

##### 1. RAG Evaluation :  En fonction des résultats, implémenter un prompt et Spacy pour les Entités Nommées



##### 2. Fast API

##### 3. Azure Deployment